In [2]:
import tensorflow as tf

tf.config.list_physical_devices('GPU')

[]

In [1]:
from dataset_loader import DatasetLoader
import matplotlib.pyplot as plt

datasetLoader = DatasetLoader('./Dataset')

In [2]:
# AUG_CONF = {'flip': [('horizontal',0.3),
#                      ('vertical', 0.5)],
#             'rotate_ccw': 0.2,
#             'rotate_cw': 0.7,
#             'translate':[(30, 0.1),
#                          (20, 0.3)]}
train_ds, val_ds, test_ds = datasetLoader.prepare_dataset(flip='horizontal',
                                                          rotate='ccw',
                                                          translate=30)

Importing the datasets with the following parameters...
   Dataset path                    : ./Dataset
   Train-Val dataset ratio        : 0.7

Splitting training and validation sets...
num of training data:  2575
num of validation data:  1105
num of testing data:  3710


In [ ]:
train_ds

In [ ]:
num_samples = sum(1 for _ in train_ds)
print("Number of samples in train_ds:", num_samples)

In [ ]:
for img_batch, label_batch in train_ds.take(1):
    img_batch = img_batch.numpy()
    label_batch = label_batch.numpy()

    batch_size = img_batch.shape[0] 

    fig, axes = plt.subplots(batch_size, 2, figsize=(8, batch_size * 3))
    
    for i in range(batch_size):
        axes[i, 0].imshow(img_batch[i])
        axes[i, 0].axis("off")
        axes[i, 0].set_title("Image")


        axes[i, 1].imshow(label_batch[i].squeeze(), cmap="gray")
        axes[i, 1].axis("off")
        axes[i, 1].set_title("Label")

    plt.tight_layout()
    plt.show()

In [7]:
from model import *
from tensorflow.keras.layers import Input

model = UNet(3, 0.2)

In [8]:
model(Input(shape=(128,128,3)))


<KerasTensor shape=(None, 128, 128, 3), dtype=float32, sparse=False, name=keras_tensor_15>

In [9]:
model.summary()

Model: "UNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_1 (DownBlock)           │ ?                      │        39,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_2 (DownBlock)           │ ?                      │       222,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_3 (DownBlock)           │ ?                      │       887,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_4 (DownBlock)           │ ?                      │     3,544,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_5 (DownBlock)           │ ?                      │    14,166,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_1 (UpBlock)             │ ?                      │     8,262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_2 (UpBlock)             │ ?                      │     2,066,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_3 (UpBlock)             │ ?                      │       517,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_4 (UpBlock)             │ ?                      │       112,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 128, 128, 3)    │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,819,203 (113.75 MB)

 Trainable params: 29,807,427 (113.71 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [11]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
from tensorflow.keras.metrics import IoU
from losses import CustomLoss

LOSS_CONF = {'dice_loss': 0.2,
             'iou_loss': 0.8,
             'focal_loss': 0}

loss = CustomLoss(LOSS_CONF)
optimizer = Adam(learning_rate=0.001)
iou_metrics = IoU(3, [0])

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'], run_eagerly=True)

history = model.fit(train_ds, validation_data=val_ds, epochs=3, verbose = 1)


Epoch 1/3


      5/Unknown 35s 7s/step - acc: 0.1956 - loss: 0.8376

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *

def dice_coef(y_true, y_pred, smooth=1):
    numerator = 2*tf.reduce_sum(y_true * y_pred) + smooth
    denominator = tf.reduce_sum(y_true + y_pred) + smooth

    return numerator/denominator

a = tf.ones([3,4])
b = tf.random.normal([3,4])

print(dice_coef(a,b))